In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns

In [3]:
sns.set(style="ticks", color_codes=True)
iris = sns.load_dataset("iris")

In [4]:
type(iris)

pandas.core.frame.DataFrame

In [5]:
from math import log
import operator

In [6]:
def createDataSet():
    dataSet = [[1,1,'yes'],
              [1,1,'yes'],
              [1,0,'no'],
              [0,1,'no'],
              [0,1,'no']]
    labels = ['no surfing', 'flippers']
    return dataSet, labels

In [14]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts={}
    for featVec in dataSet:
        currentLabel = featVec[-1]
#         print(labelCounts)
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt=0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries # numEntries : 5
        shannonEnt -=prob*log(prob,2)
#         print(shannonEnt)
    return shannonEnt

In [78]:
myDat, labels = createDataSet()
calcShannonEnt(myDat)

0.9709505944546686

In [10]:
def splitDataSet(dataSet, axis, value):
    retDataSet=[]
    for featVec in dataSet:
        if featVec[axis]==value:
#             print(featVec)
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [11]:
splitDataSet(myDat,0,0)

[[1, 'no'], [1, 'no']]

In [53]:
splitDataSet(myDat,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [54]:
splitDataSet(myDat,1,0)

[[1, 'no']]

In [55]:
splitDataSet(myDat,1,1)

[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]

In [25]:
print(myDat)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]


In [71]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0])-1 # 2개 ([1,1])
#     print(numFeatures)
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
#         print(dataSet)
        featList = [example[i] for example in dataSet]
#         print(featList)
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
#             print(subDataSet)
            prob = len(subDataSet)/ float(len(dataSet))
            newEntropy +=prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if (infoGain>bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature


In [72]:
chooseBestFeatureToSplit(myDat) # 0이 나오는데 첫번째 속성으로 나누는게 제일 좋다고 말하는거야

0

In [74]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] +=1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [75]:
##### 얘가 제일 중요해!!!!!!!!!!!!!!!!!!!#####
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value]= createTree(splitDataSet (dataSet, bestFeat, value), subLabels)
    return myTree

In [84]:
myDat, labels = createDataSet()

In [85]:
print(myDat)
print(labels)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['no surfing', 'flippers']


In [82]:
print(createTree(myDat, labels))

{'no surfing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}


In [42]:
def retrieveTree(i):
    listOfTrees = [{'no surfing':{0: 'no', 1:{'flippers':{0:'no',1:'yes'}}}},
                  {'no surfing':{0: 'no', 1:{'flippers':{0:{'head':{0:'no',1:'yes'}},1:'no'}}}}]
    return listOfTrees[i]

In [59]:
def classify(inputTree,featLabels,testVec):
    firstStr= list(inputTree)[0]
#     print(inputTree)
#     print(firstStr)
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat=secondDict[key]
    if isinstance(valueOfFeat,dict):
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else:
        classLabel = valueOfFeat
    return classLabel

In [57]:
myTree = retrieveTree(0)
print(myTree['no surfing'][1]['flippers'][0])

no


In [60]:
classify(myTree,labels,[1,1])

'yes'

In [46]:
classify(myTree,labels,[0,1])

{'no surfing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
no surfing


'no'

In [47]:
classify(myTree,labels,[1,0])

{'no surfing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
no surfing
{'flippers': {0: 'no', 1: 'yes'}}
flippers


'no'